In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
g=open('C:/Users/jiang/Desktop/rfm_data.csv',encoding='gbk')
sale_data=pd.read_csv(g)
g.close()

In [3]:
sale_data.head()

,buy_time,city_code,customer_code,bill_code,sale_amt
0,2019-09-25,QB,16449,9440052,282.00
1,2019-09-25,QB,16449,9440696,7096.48
2,2019-09-25,QB,16449,9438206,3017.80
3,2019-09-25,QB,16449,9440583,6411.00
4,2019-09-25,QB,16449,9439816,80.00


In [4]:
sale_data.dtypes

buy_time          object
city_code         object
customer_code      int64
bill_code          int64
sale_amt         float64
dtype: object

In [5]:
len(sale_data)

174409

In [11]:
sale_data.drop_duplicates(subset=['buy_time','city_code','customer_code'],keep='first',inplace=True)

In [12]:
len(sale_data)

83397

In [13]:
sale_data.isnull().any()

buy_time         False
city_code        False
customer_code    False
bill_code        False
sale_amt         False
date_diff        False
dtype: bool

In [14]:
import time
from datetime import datetime

In [15]:
def time_diff(x):
    x=x[0]
    day_diff=(pd.to_datetime('today')-x).days
    return day_diff

sale_data['buy_time']=pd.to_datetime(sale_data['buy_time'])

sale_data['date_diff']=sale_data.apply(time_diff,axis=1)

In [16]:
sale_data.head()

,buy_time,city_code,customer_code,bill_code,sale_amt,date_diff
0,2019-09-25,QB,16449,9440052,282.00,180
10,2019-09-25,QB,19658,9441255,3555.70,180
11,2019-09-25,QB,22813,9437931,1459.54,180
17,2019-09-25,QB,31295,9438367,160.00,180
19,2019-09-25,QB,46575,9439562,686.12,180


In [18]:
R_data = sale_data.groupby(by=['city_code','customer_code'])['date_diff']
R_Agg= R_data.agg([('最近一次消费','min')])

In [22]:
F_data = sale_data.groupby(by=['city_code','customer_code'])['bill_code']
F_Agg= F_data.agg([('消费频次','count')])

In [24]:
M_data =sale_data.groupby(by=['city_code','customer_code'])['sale_amt']
M_Agg = M_data.agg([('消费金额',sum)])

In [25]:
rfm = R_Agg.join(F_Agg).join(M_Agg)
rfm.head()

最近一次消费  消费频次       消费金额
city_code customer_code                         
QB        16449               6   104  585536.80
          19658               4    75  345461.63
          21318              89     7    5522.47
          21345              49    10   29378.35
          22813               4    73  640674.40

In [26]:
rfm=rfm.reset_index(drop = False)

In [27]:
rfm.head()

,city_code,customer_code,最近一次消费,消费频次,消费金额
0,QB,16449,6,104,585536.80
1,QB,19658,4,75,345461.63
2,QB,21318,89,7,5522.47
3,QB,21345,49,10,29378.35
4,QB,22813,4,73,640674.40


In [ ]:
给客户进行分层，按照1-5的等级进行打标签

In [28]:
bins = rfm.最近一次消费.quantile(q=np.linspace(0,1,6),interpolation='nearest')  
bins[0] =0  
labels =[5, 4, 3, 2, 1]  
R1 = pd.cut(rfm.最近一次消费,bins,labels=labels);
bins = rfm.消费频次.quantile(q=np.linspace(0,1,6),interpolation='nearest')  
bins[0]=0 
labels=[1,2,3,4,5] 
F1 = pd.cut(rfm.消费频次,bins,labels=labels);
bins = rfm.消费金额.quantile(q=np.linspace(0,1,6),interpolation='nearest')  
bins[0]=0  
labels=[1, 2, 3, 4, 5]  
M1 = pd.cut(rfm.消费金额,bins,labels=labels);
rfm['R1']=R1  
rfm['F1']=F1  
rfm['M1']=M1
rfm.head()

,city_code,customer_code,最近一次消费,消费频次,消费金额,R1,F1,M1
0,QB,16449,6,104,585536.80,4,5,5
1,QB,19658,4,75,345461.63,5,5,5
2,QB,21318,89,7,5522.47,1,2,2
3,QB,21345,49,10,29378.35,3,2,3
4,QB,22813,4,73,640674.40,5,5,5


In [29]:
rfm['RFM'] =0.2*R1.astype(int) + 0.3*F1.astype(int) + 0.5*M1.astype(int)  
rfm.head()

,city_code,customer_code,最近一次消费,消费频次,消费金额,R1,F1,M1,RFM
0,QB,16449,6,104,585536.80,4,5,5,4.8
1,QB,19658,4,75,345461.63,5,5,5,5.0
2,QB,21318,89,7,5522.47,1,2,2,1.8
3,QB,21345,49,10,29378.35,3,2,3,2.7
4,QB,22813,4,73,640674.40,5,5,5,5.0


In [34]:
bins = rfm.RFM.quantile(q=[0,0.125,0.25,0.375,0.5,0.625,0.75,0.875,1],interpolation='nearest')  
bins[0] =0
labels =['流失客户','一般维持客户','一般发展客户','潜力客户','重要挽留客户','重要保持客户','重要发展客户','重要价值客户']
rfm['用户分层'] = pd.cut(rfm.RFM,bins,labels=labels)


rfm=rfm.rename(columns={'最近一次消费':'last_sale_day','消费频次':'sale_frq','消费金额':'sale_amt','用户分层':'customer_classification',})

In [35]:
rfm.head()

,city_code,customer_code,last_sale_day,sale_frq,sale_amt,R1,F1,M1,RFM,customer_classification
0,QB,16449,6,104,585536.80,4,5,5,4.8,重要价值客户
1,QB,19658,4,75,345461.63,5,5,5,5.0,重要价值客户
2,QB,21318,89,7,5522.47,1,2,2,1.8,一般维持客户
3,QB,21345,49,10,29378.35,3,2,3,2.7,潜力客户
4,QB,22813,4,73,640674.40,5,5,5,5.0,重要价值客户


In [38]:
#汇总数据
df1=rfm.reset_index().dropna()

In [42]:
df1=df1.pivot_table('customer_code',index='customer_classification',aggfunc='count')

In [43]:
df1

,customer_code
customer_classification,
流失客户,458
一般维持客户,379
一般发展客户,470
潜力客户,358
重要挽留客户,391
重要保持客户,473
重要发展客户,335
重要价值客户,403


In [44]:
df1=df1.rename(columns={'customer_code':'人数'}).reset_index()

In [45]:
df1

,customer_classification,人数
0,流失客户,458
1,一般维持客户,379
2,一般发展客户,470
3,潜力客户,358
4,重要挽留客户,391
5,重要保持客户,473
6,重要发展客户,335
7,重要价值客户,403
